In [2]:
import django, os, sys, time, resource, re, gc, shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from django.db.models import Count, Avg, F, Sum
import math

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from utils.utils import ars

qid = 1558
q = Query.objects.get(pk=qid)

In [74]:

p = Project.objects.get(title="NETs")
pdocs = Doc.objects.filter(query__project=p)

techs = Technology.objects.filter(project=p).exclude(name__in=["Bioenergy","Storage","Blue Carbon"])

techs = Technology.objects.filter(project=p).exclude(name__in=["Blue Carbon"])

tdocs = set(list(Doc.objects.filter(
    technology__in=techs
).values_list('UT',flat=True)))

qtdocs = set(list(Doc.objects.filter(
    query__technology__in=techs
).values_list('UT',flat=True)))

## These are all docs in the project

pdocs =  Doc.objects.filter(
    UT__in=tdocs.union(qtdocs)
)

print(pdocs.distinct().count())#techs = Technology.objects.filter(project=p)


7604


In [78]:
queries = Query.objects.filter(technology__in=techs,doc__isnull=False,type="default").distinct()

qts = queries.annotate(
    ndocs = Count('doc')
)

qtdf = pd.DataFrame.from_dict(list(qts.sort_by('id').values('id','technology__name','ndocs','title','text','database')))

qtdf.head()

AttributeError: 'QuerySet' object has no attribute 'sort'

In [76]:
trdocs = set(list(pdocs.filter(
    docownership__relevant=1,
    docownership__query__technology__in=techs
).values_list('UT',flat=True)))

ttdocs = set(list(Doc.objects.filter(
    technology__in=techs
).values_list('UT',flat=True)))


allreldocs =  Doc.objects.filter(
    UT__in=trdocs.union(ttdocs)
)

print(allreldocs.count())

1997


In [87]:
global unfound
unfound = 0

global multmatches
multmatches = 0

global found
found = 0

global indb
indb = 0

spapers = pd.read_csv('../tables/unique_spreadsheet_papers.csv')


def find_paper(x):
    
    try:
    
        tis = x['X3'].split()
        
    except:
        print(x['X3'])
        tis = str(x['X3']).split()
    
    if len(tis) > 8:
        ti = ' '.join(tis[:8])
    else:
        ti = x['X3']
        
    docs = pdocs.filter(title__icontains=ti)
    
    global unfound
    global found
    global multmatches
    
    if docs.count() == 1:
    
        doc = docs.first()
        found +=1
    elif docs.count() > 1:
        doc = "None"
        multmatches+=1
    else:       
        di = x['X3'].replace('http://dx.doi.org/','')
        docs = pdocs.filter(wosarticle__di=di)
        if docs.count()==1:
            doc = docs.first()
            found+=1
        elif docs.count() > 1:
            doc = "None"
            multmatches+=1
        else:
            au = x['X1'].split()[0].split(',')[0]
            docs = pdocs.filter(docauthinst__AU__icontains=au,docauthinst__position=1,PY=x['X2']).distinct()
            if docs.count()==1:
                doc = docs.first()
                found+=1
            elif docs.count() > 1:
                doc = "None"
                multmatches+=1
            else:
                ti = re.findall('[A-Z]{1}\.{1}[^;]([^\.]{5,})',x['X3'])
                if len(ti) > 0:
                    ti = ti[0]
                else:
                    ti = "zwasdfölkjag"
                docs = pdocs.filter(title__icontains=ti)
                if docs.count()==1:
                    doc = docs.first()
                    found+=1
                else:
                    doc = "None"
                    unfound+=1
    
    return doc

spapers['doc_id'] = spapers.apply(lambda x: find_paper(x), axis=1)

print("no matches: {}".format(unfound))
print("multiple matches: {}".format(multmatches))
print("found {}".format(found))


spapers.head()

nan
no matches: 46
multiple matches: 15
found 800


,Unnamed: 0,X1,X2,X3,technology,doc_id
0,1,"Renforth, P",2012.0,The potential of enhanced weathering in the UK,Enhanced weathering (terrestrial and ocean),WOS:000309790500021
1,3,Schuiling & Krijgsman,2006.0,ENHANCED WEATHERING: AN EFFECTIVE AND CHEAP TO...,Enhanced weathering (terrestrial and ocean),WOS:000237646500016
2,4,Köhler et al.,2010.0,Geoengineering potential of artificially enhan...,Enhanced weathering (terrestrial and ocean),WOS:000284529000019
3,5,Taylor et al.,2016.0,Enhanced weathering strategies for stabilizing...,Enhanced weathering (terrestrial and ocean),WOS:000373060000019
4,6,Hangx & Spiers,2009.0,Coastal spreading of olivine to control atmosp...,Enhanced weathering (terrestrial and ocean),WOS:000272373000010


In [18]:
found

360

In [90]:
spapers[spapers['doc_id'] == "None"].reset_index()['X3']


notfound = spapers[spapers['doc_id'] == "None"].reset_index()

notfou


#spapers[spapers['doc_id'] == "None"].groupby('technology').count()

# Go through docs, mark include1 if in allreldocs, mark include2 if in list generated in R


#spapers[spapers['doc_id'] == "None"].reset_index()['X3'][18]




,index,Unnamed: 0,X1,X2,X3,technology,doc_id
0,21,23,"Holmes, I., Kirby, K.R., Potvin, C.",2016.0,Agroforestry within REDD+: experiences of an i...,Afforestation and Reforestation,None
1,34,36,"Xiaoming Feng, Bojie Fu, Shilong Piao, Shuai W...",2016.0,Revegetation in China's Loess Plateau is appro...,Afforestation and Reforestation,None
2,69,71,"Xiao, JF",2014.0,Satellite evidence for significant biophysical...,Afforestation and Reforestation,None
3,108,110,"Arora, VK, Montenegro, A",2011.0,Small temperature benefits provided by realist...,Afforestation and Reforestation,None
4,128,130,"Lenton, TM",2010.0,The potential for land-based biological CO2 re...,Afforestation and Reforestation,None
5,132,136,"Xu, B, Guo, ZD, Piao, SL, Fang, JY",2010.0,Biomass carbon stocks in China’s forests betwe...,Afforestation and Reforestation,None
6,134,138,"Gorte, R.W.",2010.0,Carbon sequestration in forests,Afforestation and Reforestation,None
7,150,154,"Henry, M, Tittonell, P, Manlay, RJ, Bernoux, M...",2009.0,"Biodiversity, carbon stocks and sequestration ...",Afforestation and Reforestation,None
8,196,202,"Glenday, J",2006.0,Carbon storage and emissions offset potential ...,Afforestation and Reforestation,None
9,213,219,Melvin G.R. Cannell,2003.0,Carbon sequestration and biomass energy offset...,Afforestation and Reforestation,None


In [33]:
for q in queries[:10]:
    ds = q.doc_set.all()
    dids = list(q.doc_set.all().values_list('UT',flat=True))
    ds = Doc.objects.filter(UT__in=dids)
    print(ds.count())
    ds = ds.annotate(
        qs = Count('query')
    )
    print(ds.filter(qs__lt=4).count())
    
    print(q.title)

68
0
enhanced_weathering_wos
2
0
forests1_backward_1_1
3
0
*7B_BlueDemos
101
0
enhanced_weathering_wos_2
637
0
soil_carbon_sequestration_scopus
4
0
DAC_extra
422
9
synonyms_all
660
0
soil_carbon_sequestration
396
0
beccs_scopus
391
6
synonyms_scopus


In [25]:
q = queries.first()

ds = q.doc_set.all()

print(ds.count())

ds = ds.annotate(
    qs = Count('query')
)

print(ds.filter(qs__lt=2).count())

q.title

198
198


'NETs_scoping_DAC'

In [ ]:


trdocs = set(list(pdocs.filter(
    docownership__relevant=1,
    docownership__query__technology__in=techs
).values_list('UT',flat=True)))

ttdocs = set(list(Doc.objects.filter(
    technology__in=techs
).values_list('UT',flat=True)))

print(len(trdocs))
print(len(ttdocs))

allreldocs =  Doc.objects.filter(
    UT__in=trdocs.union(ttdocs)
)

print(allreldocs.count())

print(allreldocs.order_by('PY').first().PY)
print(allreldocs.order_by('PY').first().title)

print(allreldocs.filter(PY__lt=1990).count())